In [71]:
import sys
import re
from enum import Enum
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
import heapq

In [72]:
class CellType(Enum):
    EMPTY = 0
    BLOCK = 1
    SOURCE = 2
    TARGET = 3
    VIA = 4

@dataclass
class Cell:
    kind: CellType
    cost: int = 0

@dataclass
class Coord:
    row: int
    col: int
    layer: int

In [ ]:
class MazeRouter:
    NUM_LAYERS = 2
    def __init__(self, in_path: str):
        self.in_path = in_path
        self.min_cost = 0
        self.cols = 0
        self.rows = 0
        self.Direction = 0
        self.via = 0

        self.grid: List[List[List[Cell]]] = []
        self.nets: List[List[Coord]] = []
        self.same_layer_nets: List[List[Coord]] = []
        self.multi_layer_nets: List[List[Coord]] = []
        self.prev: Dict[Tuple[int,int,int], Optional[Tuple[int,int,int]]] = {}

        self.load_config()
        self.categorize_nets()

    def load_config(self):
        with open(self.in_path, 'r') as f:
            header = f.readline().strip()
            r, c, b, v = map(int, header.split(','))
            self.rows, self.cols = r, c
            self.Direction, self.via = b, v
            self.grid = [
                [[Cell(CellType.EMPTY) for _ in range(self.cols)] for _ in range(self.rows)]
                for _ in range(self.NUM_LAYERS)
            ]
            for line in f:
                line = line.strip()
                if line.startswith('OBS'):
                    self.add_obstacle(line)
                elif line.startswith('net'):
                    self.add_net(line)

    def add_obstacle(self, text: str):
        m = re.match(r"OBS\s*\((\d+),\s*(\d+),\s*(\d+)\)", text)
        if not m:
            print('Incorrect Format.', file=sys.stderr)
            return
        layer, row, col = map(int, m.groups())
        if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
            self.grid[layer][row][col].kind = CellType.BLOCK

    def add_net(self, text: str):
        path: List[Coord] = []
        for m in re.finditer(r"\((\d+),\s*(\d+),\s*(\d+)\)", text):
            L, r, c = map(int, m.groups())
            path.append(Coord(r, c, L))
        if len(path) >= 2:
            self.nets.append(path)

    def categorize_nets(self):
        for net in self.nets:
            layers = {p.layer for p in net}
            if len(layers) == 1:
                self.same_layer_nets.append(net)
            else:
                self.multi_layer_nets.append(net)

    @staticmethod
    def _is_preferred_move(dr: int, dc: int, layer: int) -> bool:
        return (dr == 0 and layer == 0) or (dc == 0 and layer == 1)

    def _reset_costs(self):
        INF = float('inf')
        for L in range(self.NUM_LAYERS):
            for r in range(self.rows):
                for c in range(self.cols):
                    cell = self.grid[L][r][c]
                    cell.cost = -1 if cell.kind == CellType.BLOCK else INF
        return INF

    def _lee_wave(self, src: Coord, tgt: Coord, L: int) -> bool:
        INF = self._reset_costs()
        self.grid[L][src.row][src.col].cost = 0
        pq: List[Tuple[float,int,int]] = [(0, src.row, src.col)]
        while pq:
            acc, r, c = heapq.heappop(pq)
            if acc > self.grid[L][r][c].cost:
                continue
            if (r, c) == (tgt.row, tgt.col):
                self.min_cost = acc
                return True
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = r+dr, c+dc
                if 0 <= nr < self.rows and 0 <= nc < self.cols:
                    neigh = self.grid[L][nr][nc]
                    if neigh.kind != CellType.BLOCK:
                        step = 1 if self._is_preferred_move(dr, dc, L) else self.Direction
                        new_cost = acc + step
                        if new_cost < neigh.cost:
                            neigh.cost = new_cost
                            heapq.heappush(pq, (new_cost, nr, nc))
        return False

    def _lee_wave_3d(self, src: Coord, tgt: Coord) -> bool:
        INF = self._reset_costs()
        self.prev.clear()
        start = (src.layer, src.row, src.col)
        self.grid[src.layer][src.row][src.col].cost = 0
        self.prev[start] = None
        pq: List[Tuple[float,int,int,int]] = [(0, src.layer, src.row, src.col)]
        while pq:
            acc, L, r, c = heapq.heappop(pq)
            if acc > self.grid[L][r][c].cost:
                continue
            if (L, r, c) == (tgt.layer, tgt.row, tgt.col):
                self.min_cost = acc
                return True
        
            for dr, dc in [(0,-1),(0,1),(-1,0),(1,0)]:
                if not self._is_preferred_move(dr, dc, L):
                    continue
                nr, nc = r+dr, c+dc
                if 0 <= nr < self.rows and 0 <= nc < self.cols:
                    neigh = self.grid[L][nr][nc]
                    if neigh.kind != CellType.BLOCK:
                        cost = acc + 1
                        if cost < neigh.cost:
                            neigh.cost = cost
                            self.prev[(L,nr,nc)] = (L,r,c)
                            heapq.heappush(pq, (cost, L, nr, nc))

            other = 1 - L
            via_cell = self.grid[other][r][c]
            if via_cell.kind != CellType.BLOCK:
                cost = acc + self.via
                if cost < via_cell.cost:
                    via_cell.cost = cost
                    self.prev[(other,r,c)] = (L,r,c)
                    heapq.heappush(pq, (cost, other, r, c))

            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                if self._is_preferred_move(dr, dc, L):
                    continue
                nr, nc = r+dr, c+dc
                if 0 <= nr < self.rows and 0 <= nc < self.cols:
                    neigh = self.grid[L][nr][nc]
                    if neigh.kind != CellType.BLOCK:
                        cost = acc + self.Direction
                        if cost < neigh.cost:
                            neigh.cost = cost
                            self.prev[(L,nr,nc)] = (L,r,c)
                            heapq.heappush(pq, (cost, L, nr, nc))
        return False

    def _back_propagate_2d(self, src: Coord, tgt: Coord) -> List[Coord]:
        path = [tgt]
        cur = tgt
        iters = 0
        max_iters = self.rows*self.cols*2
        while (cur.row, cur.col) != (src.row, src.col) and iters < max_iters:
            iters += 1
            best = None
            best_cost = float('inf')
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = cur.row+dr, cur.col+dc
                if 0 <= nr < self.rows and 0 <= nc < self.cols:
                    cell = self.grid[cur.layer][nr][nc]
                    if cell.cost>=0 and (cell.cost<best_cost or (cell.cost==best_cost and self._is_preferred_move(cur.row-nr, cur.col-nc, cur.layer))):
                        best_cost = cell.cost
                        best = Coord(nr,nc,cur.layer)
            if not best:
                break
            cur = best
            path.insert(0,cur)
        return path

    def _back_propagate_3d(self, src: Coord, tgt: Coord) -> List[Coord]:
        path: List[Coord] = []
        key = (tgt.layer, tgt.row, tgt.col)
        while key is not None:
            L, r, c = key
            path.append(Coord(r,c,L))
            key = self.prev.get(key)
        path.reverse()
        return path

    def _net_score_same(self, net: List[Coord]) -> int:
        rows = [p.row for p in net]
        cols = [p.col for p in net]
        return (max(rows)-min(rows)+1) * (max(cols)-min(cols)+1)

    def _net_score_multi(self, net: List[Coord]) -> int:
        src, tgt = net[0], net[1]
        return abs(src.row - tgt.row) + abs(src.col - tgt.col) + abs(src.layer - tgt.layer)

    def route_same_layer_multi_targets(self):
        for net in self.same_layer_nets:
            for coord in net:
                self.grid[coord.layer][coord.row][coord.col].kind = CellType.BLOCK
                
        scores = [(self._net_score_same(net), idx, net) for idx, net in enumerate(self.same_layer_nets)]
        scores.sort(key=lambda x: x[0])
        results_map = {}
        for _, idx, net in scores:
            layer = net[0].layer
            connected = [net[0]]
            targets = net[1:].copy()
            segments = []
            
            self.grid[net[0].layer][net[0].row][net[0].col].kind = CellType.SOURCE
            for tgt in targets:
                self.grid[tgt.layer][tgt.row][tgt.col].kind = CellType.TARGET
            
            while targets:
                best_seg = None
                for start in connected:
                    for tgt in targets:
                        if self._lee_wave(start, tgt, layer):
                            path = self._back_propagate_2d(start, tgt)
                            cost = self.min_cost
                        else:
                            if not self._lee_wave_3d(start, tgt):
                                continue
                            path = self._back_propagate_3d(start, tgt)
                            cost = self.min_cost
                        if len(path)>1 and (best_seg is None or cost<best_seg[0]):
                            best_seg = (cost, start, tgt, path)
                if not best_seg:
                    break
                cost, start, chosen, path = best_seg
                segments.append((start, chosen, path, cost))
                for c in path:
                    if all((c.row!=cc.row or c.col!=cc.col or c.layer!=cc.layer) for cc in connected):
                        self.grid[c.layer][c.row][c.col].kind = CellType.BLOCK
                        connected.append(c)
                targets.remove(chosen)
            total = sum(s[3] for s in segments)
            results_map[idx] = (idx, segments, total)
            
            for coord in net:
                self.grid[coord.layer][coord.row][coord.col].kind = CellType.BLOCK
            

        return [results_map[i] for i in range(len(self.same_layer_nets))]

    def route_multi_layer(self):

        for net in self.same_layer_nets:
            for coord in net:
                self.grid[coord.layer][coord.row][coord.col].kind = CellType.BLOCK

        
        scores = [
            (self._net_score_multi(net), idx, net)
            for idx, net in enumerate(self.multi_layer_nets)
        ]
        scores.sort(key=lambda x: x[0])

        results: List[Tuple[int, Coord, List[Coord], List[List[Coord]], float]] = []

        for _, idx, net in scores:
            src = net[0]
            targets = net[1:].copy()
            connected = [src]
            segment_paths: List[List[Coord]] = []
            total_cost = 0.0


            self.grid[src.layer][src.row][src.col].kind = CellType.SOURCE
            for tgt in targets:
                self.grid[tgt.layer][tgt.row][tgt.col].kind = CellType.TARGET


            while targets:
                best_seg = None
                for start in connected:
                    for tgt in targets:
                        if not self._lee_wave_3d(start, tgt):
                            continue
                        path = self._back_propagate_3d(start, tgt)
                        cost = self.min_cost
                        if len(path) > 1 and (best_seg is None or cost < best_seg[0]):
                            best_seg = (cost, start, tgt, path)
                if not best_seg:
                    break  

                cost, start, chosen_tgt, path = best_seg
                segment_paths.append(path)
                total_cost += cost

           
                for cell in path:
                    if all((cell.row != p.row or cell.col != p.col or cell.layer != p.layer)
                           for p in connected):
                        self.grid[cell.layer][cell.row][cell.col].kind = CellType.BLOCK
                        connected.append(cell)

            
                targets.remove(chosen_tgt)

        
            for coord in net:
                self.grid[coord.layer][coord.row][coord.col].kind = CellType.BLOCK

            net_targets = net[1:]
            results.append((idx, src, net_targets, segment_paths, total_cost))

      
        results.sort(key=lambda x: x[0])
        return results


In [ ]:
def main():
    router = MazeRouter('input8_f.txt')
    print(f"Grid: {router.rows}×{router.cols}, DirPenalty={router.Direction}, viaCost={router.via}\n")

    print("Obstacles:")
    for L in range(MazeRouter.NUM_LAYERS):
        for r in range(router.rows):
            for c in range(router.cols):
                if router.grid[L][r][c].kind == CellType.BLOCK:
                    print(f"  layer{L}, row{r}, col{c}")
    print()

    print("Loaded same-layer nets:")
    for i, net in enumerate(router.same_layer_nets):
        pts = [(p.layer, p.row, p.col) for p in net]
        print(f"  net{i}: {pts}")
    print()

    print("=== MULTI-TARGET SAME-LAYER ROUTES ===")
    for idx, segments, total in router.route_same_layer_multi_targets():
        print(f"net{idx}: total_cost = {total}")
        for start, tgt, path, cost in segments:
            coords = [(c.layer, c.row, c.col) for c in path]
            print(f"  from {(start.layer, start.row, start.col)} "
                  f"→ {(tgt.layer, tgt.row, tgt.col)}: path={coords}, cost={cost}")
        if not segments:
            print("  (no route found)")
        print()

    print("=== MULTI-LAYER ROUTES ===")
    for idx, src, targets, segment_paths, total_cost in router.route_multi_layer():
        print(f"net{idx}: total_cost = {total_cost}")
        for path in segment_paths:
            start, end = path[0], path[-1]
            coords = [(c.layer, c.row, c.col) for c in path]
            print(f"  from {(start.layer, start.row, start.col)} "
                  f"→ {(end.layer, end.row, end.col)}: path={coords}")
        print()

if __name__ == '__main__':
    main()


Grid: 8×8, DirPenalty=3, viaCost=5

Obstacles:
  layer0, row0, col3
  layer0, row0, col4
  layer0, row3, col5
  layer0, row3, col6
  layer1, row3, col1
  layer1, row4, col1
  layer1, row4, col4
  layer1, row5, col1

Loaded same-layer nets:

=== MULTI-TARGET SAME-LAYER ROUTES ===
=== MULTI-LAYER ROUTES ===
net0: total_cost = 27.0
  from (0, 0, 1) → (0, 0, 7): path=[(0, 0, 1), (0, 0, 2), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7), (0, 0, 7)]
  from (0, 1, 6) → (1, 3, 6): path=[(0, 1, 6), (1, 1, 6), (1, 2, 6), (1, 3, 6)]
  from (1, 3, 6) → (1, 5, 4): path=[(1, 3, 6), (1, 4, 6), (1, 5, 6), (1, 5, 5), (1, 5, 4)]

net1: total_cost = 22.0
  from (1, 1, 1) → (0, 3, 3): path=[(1, 1, 1), (1, 2, 1), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 3, 3)]
  from (1, 2, 1) → (1, 7, 1): path=[(1, 2, 1), (1, 2, 0), (1, 3, 0), (1, 4, 0), (1, 5, 0), (1, 6, 0), (1, 7, 0), (1, 7, 1)]

